# https://github.com/Arie-dev/Web-Scraping



#### Web Scraping and File Upload to MinIO

This script performs web scraping using Beautiful Soup and uploads the scraped data to a MinIO object storage service.

##### Dependencies
- `BeautifulSoup` from `bs4`: Used for parsing HTML and XML documents.
- `requests`: Used for making HTTP requests.
- `Minio`: Client for interacting with the MinIO storage service.
- `BytesIO`: For handling byte streams in memory.
- `datetime`: For working with date and time.

##### Setup

To use this script, ensure that the following dependencies are installed:
```bash
pip install beautifulsoup4 requests minio
```

or 

```bash
pip install -r requirements.txt
```

Also, make sure you have a running MinIO instance. This script connects to a MinIO server running on `localhost:9000` with the specified access and secret keys.

###### Usage

1. **Initialize the MinIO client** by calling the `setup_minio_client` function.
2. The script checks if the bucket named 'bronze' exists and creates it if it does not.
3. You can extend this script by adding functions for web scraping and uploading data to the 'bronze' bucket.

##### Example Code

In [32]:
from bs4 import BeautifulSoup
import requests
from minio import Minio
from io import BytesIO
from datetime import datetime

def setup_minio_client():
    minio_client = Minio('localhost:9000',
                         access_key='ROOTUSER',
                         secret_key='DATAINCUBATOR',
                         secure=False)
    if not minio_client.bucket_exists('bronze'):
        minio_client.make_bucket('bronze')
        print("Bucket 'bronze' created successfully")
    return minio_client
minio_client = setup_minio_client()



#### Web Scraping Function for Book Data

This function scrapes book information from the website [Books to Scrape](https://books.toscrape.com).
It retrieves details such as title, price, availability, and rating for each book listed on the first page of the catalog.

##### Dependencies
- `requests`: Used to make HTTP requests to fetch web pages.
- `BeautifulSoup` from `bs4`: Used to parse HTML content.

##### Function: `scrape_books_data`

##### Description

The `scrape_books_data` function sends a GET request to the specified URL, retrieves the HTML content of the page, 
and parses it to extract book details.

##### Returns
- `List[Dict[str, str]]`: A list of dictionaries, each containing:
  - `title`: The title of the book.
  - `price`: The price of the book.
  - `availability`: The availability status of the book.
  - `rating`: The rating of the book (as a class, e.g., "One", "Two", "Three", "Four", "Five").

If the request fails, the function will print an error message and return `None`.

##### Example Usage

```python
book_data = scrape_books_data()
for book in book_data:
    print(book)
```


In [ ]:
def scrape_books_data():
    url = "https://books.toscrape.com/catalogue/page-1.html"
    response = requests.get(url)

    if response.status_code == 200:
        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')
        book_rows = soup.find_all('article', class_='product_pod')  
        
        books_data = []
        for book in book_rows:
            title = book.find('h3').find('a')['title']
            price = book.find('p', class_='price_color').text.strip()
            availability = book.find('p', class_='instock availability').text.strip()
            rating = book.find('p', class_='star-rating')['class'][1]

            books_data.append({
                'title': title,
                'price': price,
                'availability': availability,
                'rating': rating
            })

        return books_data
    else:
        print(f"Failed to fetch page, status code: {response.status_code}")
        return None

book_data = scrape_books_data()
for book in book_data:
    print(book)


In [ ]:
import requests
from bs4 import BeautifulSoup

def scrape_quotes_data():
    url = "http://quotes.toscrape.com/page/1/"
    response = requests.get(url)

    if response.status_code == 200:
        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')
        quote_blocks = soup.find_all('div', class_='quote')  
        
        quotes_data = []
        for quote in quote_blocks:
            text = quote.find('span', class_='text').text.strip()
            author = quote.find('small', class_='author').text.strip()
           

            quotes_data.append({
                'text': text,
                'author': author,
            
            })

        return quotes_data
    else:
        print(f"Failed to fetch page, status code: {response.status_code}")
        return None

quote_data = scrape_quotes_data()
if quote_data:
    for quote in quote_data:
        print(quote)



#### Save Books Data to MinIO

This function saves a list of book data to a MinIO object storage service in CSV format.
The CSV file is named with the current date to ensure uniqueness.

##### Dependencies
- `csv`: Used to write CSV files.
- `io.StringIO`: Provides an in-memory string buffer for CSV data.
- `datetime`: Used to get the current date for naming the CSV file.
- `BytesIO`: Allows the conversion of string data to byte format for uploading.

##### Function: `save_books_data_to_minio`

###### Description

The `save_books_data_to_minio` function takes a list of book data and a MinIO client instance, 
converts the book data into CSV format, and uploads it to the specified MinIO bucket.

###### Parameters
- `books_data` (`List[Dict[str, str]]`): A list of dictionaries, where each dictionary contains:
  - `title`: The title of the book.
  - `price`: The price of the book.
  - `availability`: The availability status of the book.
  - `rating`: The rating of the book.
- `minio_client` (`Minio`): An instance of the MinIO client for interacting with the MinIO server.

###### Returns
- `None`: The function does not return a value, but it prints a success or error message based on the upload outcome.

###### Example Usage

```python
if book_data:
    save_books_data_to_minio(book_data, minio_client)
```


In [ ]:
import csv
from io import StringIO
from datetime import datetime

def save_books_data_to_minio(books_data, minio_client):
    current_datetime = datetime.now().strftime('%Y%m%d')
    object_name = f'books_data_{current_datetime}.csv'

    csv_data = StringIO()
    fieldnames = ["title", "price", "availability", "rating"]
    writer = csv.DictWriter(csv_data, fieldnames=fieldnames)
    writer.writeheader()
    
    for book in books_data:
        writer.writerow(book)

    csv_data_bytes = BytesIO(csv_data.getvalue().encode('utf-8'))
    
    try:
        minio_client.put_object(
            'bronze', object_name, csv_data_bytes, len(csv_data_bytes.getvalue())
        )
        print(f"Book data saved successfully as {object_name}")
    except Exception as e:
        print("An error occurred while uploading to Minio:", e)

if book_data:
    save_books_data_to_minio(book_data, minio_client)


In [ ]:
import csv
from io import BytesIO
from datetime import datetime
from minio import Minio
from minio.error import S3Error

def save_quotes_data_to_minio(quotes_data, minio_client):
    current_datetime = datetime.now().strftime('%Y%m%d')
    object_name = f'quotes_data_{current_datetime}.csv'

    csv_data = StringIO()
    fieldnames = ["text", "author"]
    writer = csv.DictWriter(csv_data, fieldnames=fieldnames)
    writer.writeheader()
    
    for quote in quotes_data:
        writer.writerow(quote)


    csv_data_bytes = BytesIO(csv_data.getvalue().encode('utf-8'))
    try:
        
        minio_client.put_object(
            'bronze', object_name, csv_data_bytes, len(csv_data_bytes.getvalue())
        )
        print(f"Quotes data saved successfully as {object_name}")
    except S3Error as e:
        print("An error occurred while uploading to Minio:", e)


if quote_data:
    save_quotes_data_to_minio(quote_data, minio_client)

#### Now it's your turn

The website "Quotes to Scrape" features a collection of famous quotes from notable authors, thinkers, and celebrities. Each quote is accompanied by relevant tags, allowing users to explore various themes such as life, love, success, and humor. Notable figures like Albert Einstein, J.K. Rowling, and Jane Austen are featured.

You can browse more quotes on the website [here](https://quotes.toscrape.com).

#### Its scraping time

Try scraping all the quotes and authors from the quotes on the first page. Save it as a csv into a bucket on MinIO

In [37]:
# url = "https://quotes.toscrape.com/"

#### About Medallion Architecture

Medallion Architecture is a data architecture pattern commonly used in modern data engineering, particularly in cloud platforms like Databricks or in oure case miniodb. It aims to organize data into different quality levels (or layers), enabling better data management, governance, and access. The architecture is typically divided into **three layers**, symbolized by three medals:

###### 1. **Bronze Layer (Raw Data)**

- **Purpose**: The Bronze layer stores raw, unprocessed data.
- **Characteristics**: Data is ingested in its original form without much transformation. This can include structured, semi-structured, or unstructured data.
- **Typical Sources**: This layer includes logs, clickstreams, and any other data from various sources such as IoT devices, APIs, databases, or files.
- **Advantages**: 
  - Acts as the raw historical record of all data.
  - Useful for reprocessing in case of errors or when new transformations are needed.

###### 2. **Silver Layer (Cleaned Data)**

- **Purpose**: The Silver layer refines the raw data.
- **Characteristics**: Data is filtered, cleaned, and enriched with additional attributes or metadata. Business logic and validation rules are often applied here.
- **Transformations**:
  - Handling missing values
  - Normalizing data formats
  - Data joins or enrichments from other sources
- **Usage**: This layer is ideal for reporting, analytics, and as input for machine learning models.

###### 3. **Gold Layer (Business-Level Data)**

- **Purpose**: The Gold layer contains high-quality, curated datasets that are ready for business consumption.
- **Characteristics**: This data is fully processed and optimized for specific use cases, such as analytics dashboards or data marts.
- **Optimizations**: Aggregations, computations, and business logic are applied to make the data fit for direct consumption by business users or applications.
- **Benefits**:
  - Optimized for performance (e.g., querying in BI tools).
  - Ensures data accuracy and business relevance.


links:

- https://www.databricks.com/glossary/medallion-architecture
- https://dataengineering.wiki/Concepts/Medallion+Architecture